[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/V3D-jupyter/blob/main/V3D_jupyter.ipynb)

In [ ]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

%cd /content
!git clone -b dev https://github.com/camenduru/V3D
%cd /content/V3D
!mkdir /content/V3D/output

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors -d /content/V3D/ckpts -o svd_xt.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/heheyas/V3D/resolve/main/V3D.ckpt -d /content/V3D/ckpts -o V3D_512.ckpt

!pip install gradio einops fire tyro omegaconf mediapy rembg kiui invisible-watermark pytorch-lightning kornia
!pip install git+https://github.com/openai/CLIP.git open-clip-torch PyMCubes
# !pip install https://github.com/camenduru/LGM-replicate/releases/download/replicate/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl

# !python app.py

In [ ]:
%cd /content/V3D
!PYTHONPATH="." python scripts/pub/V3D_512.py --input_path /content/V3D/assets/baby_yoda.png --save --border_ratio 0.3 --min_guidance_scale 4.5 --max_guidance_scale 4.5 --output-folder /content/V3D/output

In [ ]:
%cd /content/V3D
!PYTHONPATH="." python recon/train_from_vid.py -w --sh_degree 0 --iterations 4000 --lambda_dssim 1.0 --lambda_lpips 2.0 --save_iterations 4000 --num_pts 100_000 --video V3D/output/000000.mp4

In [ ]:
%cd /content/V3D/mesh_recon
!PYTHONPATH="." python launch.py --config configs/videonvs.yaml --gpu <gpu> --train system.loss.lambda_normal=0.1 dataset.scene=<scene_name> dataset.root_dir=<output_dir> dataset.img_wh='[512, 512]'

In [ ]:
%cd /content/V3D
!PYTHONPATH="." python refine.py --mesh <your obj mesh file> --scene <your video> --num-opt 16 --lpips 1.0 --iters 500